# CS 617 - Data Mining

## Homework 2 - Problem 3

In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import display, HTML, Math

In [2]:
# Run this cell to load in our dataset. Don't worry about what it's doing.
np.random.seed(25)

salaries_raw = pd.read_csv('data_scientist_salaries-1.csv')
salaries = salaries_raw.get(['YearsCodingProf', 'Age', 'FormalEducation', 'Salary']).dropna()

def extract_years(year_str):
    if isinstance(year_str, float):
        return year_str
    if 'older' in year_str:
        years = 65
    elif 'more' in year_str:
        years = 30
    elif 'Under' in year_str:
        years = 18
    else:
        extracted = re.findall('\d+', year_str)
        try:
            lower, upper = int(extracted[0]), int(extracted[1])
        except:
            print(extracted)
        years = np.random.randint(lower, upper + 1)
    return years + np.round(np.random.normal(0, 1), 2)

salaries['Age'] = salaries['Age'].apply(extract_years)
salaries['YearsExperience'] = salaries['YearsCodingProf'].apply(extract_years)
salaries = salaries[['YearsExperience', 'Age', 'FormalEducation', 'Salary']]
salaries = salaries[(salaries['Salary'] < 500000) & (salaries['Salary'] > 1000) & (salaries['YearsExperience'] > 0)].reset_index(drop=True)
salaries['Salary'] /= 1000

In [3]:
salaries

,YearsExperience,Age,FormalEducation,Salary
0,6.37,28.39,"Master’s degree (MA, MS, M.Eng., MBA, etc.)",120.0
1,0.35,25.78,Some college/university study without earning ...,120.0
2,4.05,31.04,"Bachelor’s degree (BA, BS, B.Eng., etc.)",70.0
3,18.48,38.78,"Bachelor’s degree (BA, BS, B.Eng., etc.)",185.0
4,4.95,33.45,"Master’s degree (MA, MS, M.Eng., MBA, etc.)",125.0
...,...,...,...,...
983,5.07,25.55,"Master’s degree (MA, MS, M.Eng., MBA, etc.)",27.0
984,8.24,30.54,Associate degree,120.0
985,6.46,32.91,"Other doctoral degree (Ph.D, Ed.D., etc.)",149.0
986,16.27,64.55,"Master’s degree (MA, MS, M.Eng., MBA, etc.)",57.0


### Design matrix

In this case, we only have one feature – `'YearsExperience'`. Our design matrix would then look something like:

In [4]:
# Don't worry about this code ---
X_as_df = pd.DataFrame()
X_as_df['1'] = np.ones(salaries.shape[0]).astype(int)
X_as_df['YearsExperience'] = salaries['YearsExperience']
X_as_df
# ---

X_as_df

,1,YearsExperience
0,1,6.37
1,1,0.35
2,1,4.05
3,1,18.48
4,1,4.95
...,...,...
983,1,5.07
984,1,8.24
985,1,6.46
986,1,16.27


In [5]:
# Converting to a numpy array
X = X_as_df.values
X

array([[ 1.  ,  6.37],
       [ 1.  ,  0.35],
       [ 1.  ,  4.05],
       ...,
       [ 1.  ,  6.46],
       [ 1.  , 16.27],
       [ 1.  ,  0.56]])

This is the design matrix! ^

### Observation vector

In [6]:
y = salaries['Salary']
y

0      120.0
1      120.0
2       70.0
3      185.0
4      125.0
       ...  
983     27.0
984    120.0
985    149.0
986     57.0
987     50.0
Name: Salary, Length: 988, dtype: float64

In [7]:
y = y.values
y

array([120.    , 120.    ,  70.    , 185.    , 125.    , 113.    ,
        83.    ,  40.    , 300.    ,  60.    , 130.    ,  60.    ,
       120.    , 102.    ,  80.    , 120.    , 115.    ,  65.    ,
        75.    , 149.    , 100.    , 150.    , 135.    , 115.    ,
       125.    ,   9.8   ,  80.    ,  92.    ,  52.    , 145.    ,
       120.    , 137.    , 200.    , 140.    , 115.    , 180.    ,
       130.    , 145.    ,  72.    , 135.    ,  90.    , 160.    ,
        78.    , 120.    , 152.    , 152.    , 140.    , 100.    ,
       130.    , 100.    , 115.    , 137.    , 100.    , 106.    ,
        80.    , 104.    ,  89.    ,  83.    , 189.    , 158.    ,
       105.    ,  90.    ,  13.3   , 135.    ,  70.    , 155.    ,
       115.    ,  57.6   , 206.    , 130.    , 182.    ,  32.    ,
       116.    , 245.    ,  45.    ,  74.    , 100.    , 220.    ,
       100.    ,  82.    ,  40.    , 150.    , 147.    ,  70.    ,
       103.    ,  90.    ,  80.    ,  61.    ,  80.    , 105. 

### Making predictions

For any vector $\vec{w} \in \mathbb{R}^{2}$, we can make predictions using

$$\vec{h} = X \vec{w}$$

Let's test it out!

In [8]:
X @ np.array([80, 3])

array([ 99.11,  81.05,  92.15, 135.44,  94.85, 109.37, 106.25,  98.96,
       116.42,  86.66,  97.97,  87.23, 103.49, 108.71,  82.04, 137.  ,
        81.98,  82.73,  88.79,  91.76,  82.16, 151.52,  90.92, 119.06,
       108.8 , 100.85,  90.11,  89.9 ,  86.75, 103.4 ,  83.42, 138.05,
       141.02, 109.7 , 130.31, 173.96,  88.13, 123.23,  89.09, 105.62,
        88.61, 138.32,  84.11,  99.68, 136.31, 127.22, 104.87, 109.88,
        94.82, 130.97, 129.62,  85.34, 133.82,  89.  ,  88.01,  98.36,
        91.88, 105.8 , 161.75,  94.88, 115.1 , 114.11,  88.64,  88.73,
       117.71, 107.51, 100.1 ,  90.89,  97.37, 120.35, 134.81, 100.22,
        93.47, 113.69,  97.55,  94.97,  94.19, 108.02,  95.63,  93.77,
        95.12, 103.52, 104.75, 110.93, 104.84,  89.  ,  96.98,  82.85,
        93.08, 146.72,  83.21, 131.03, 107.87,  95.78,  96.77,  89.06,
        86.72,  97.31, 172.46,  94.13, 131.  , 103.43,  91.67,  89.51,
        91.16,  95.48, 111.08,  85.67,  98.87,  95.84,  95.66, 109.07,
      

Our goal is to get the above array as close to `y` as possible.

### Implementing the solution

We claimed that the vector $\vec{w}$ that minimizes

$$R_{sq}(\vec{w}) = \frac{1}{n} || \vec{y} - X \vec{w} ||^2$$

is

$$\vec{w}^* = (X^TX)^{-1}X^T\vec{y}$$

In [9]:
def least_squares(X, y):
    # YOUR IMPLEMENTATION IS HERE
    return np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)

In [10]:
w_star = least_squares(X, y)

In [11]:
w_star

array([80.79258094,  3.22297875])

What if I have 10 years of experience – what should I expect my salary to be?

In [12]:
np.array([1, 10]) @ w_star

113.0223684888939

Note that these match the intercept and slope using our manual formulas!

In [13]:
def correlation(x, y):
    # YOUR IMPLEMENTATION IS HERE
    numerator = np.sum((x - np.mean(x)) * (y - np.mean(y)))
    denominator_x = np.sum((x - np.mean(x))**2)
    denominator_y = np.sum((y - np.mean(y))**2)
    correlation_coefficient = numerator / np.sqrt(denominator_x * denominator_y)
    return correlation_coefficient

def slope(x, y):
    # YOUR IMPLEMENTATION IS HERE
    numerator = np.sum((x - np.mean(x)) * (y - np.mean(y)))
    denominator = np.sum((x - np.mean(x))**2)
    slope_value = numerator / denominator
    return slope_value

def intercept(x, y):
    # YOUR IMPLEMENTATION IS HERE
    m = slope(x, y)
    intercept_value = np.mean(y) - m * np.mean(x)
    return intercept_value

In [14]:
intercept(X[:, 1], y)

80.79258094057926

In [15]:
slope(X[:, 1], y)

3.222978754831467

The intercept and slope must be the same in both ways of calculation (either via closed-form formula with pseudo-inverse or formula with correlation)

Now, solve the Linear Regression with Gradient Descent! 

In [18]:
def gradient_descent_linear_regression(X, y, alpha=0.01):
    w = np.zeros(X.shape[1])
    n = len(y)
    threshold = 1e-6 
    
    while True:
        y_pred = np.dot(X, w)
#         print(y_pred)
        gradient = -(1/n) * np.dot(X.T, (y - y_pred))
#         print(gradient)
        w -= alpha * gradient
#         print(w)
        
        if np.linalg.norm(alpha * gradient) < threshold:
            break
    
    return w

coefficients = gradient_descent_linear_regression(X, y)
intercept = coefficients[0]
slopes = coefficients[1]
print(intercept)
print(slopes)

80.79234696555814
3.2229941864546667


Now, solve the Linear Regression with built-in libary sklearn. Check https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

In [17]:
# YOUR IMPLEMENTATION IS HERE
from sklearn.linear_model import LinearRegression

reg = LinearRegression().fit(X, y)

# Get the coefficients
intercept = reg.intercept_
coefficients = reg.coef_

print(intercept)
print(coefficients[-1])

80.79258094057927
3.2229787548314657
